In [3]:
import shap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from joblib import load
from data_preprocess import preprocess_data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB
None
      id  gender   age  hypertension  heart_disease ever_married  \
0   9046    Male  67.0             0              1          Yes   
1  51676  Female

In [4]:
log_reg_model = load('C:/Users/zen/Documents/-- four/s2/FYP I/XAl-on-healthcare-diagnostics/version_3.0/non-xai/logistic_regression_model.pkl')
dt_model = load('C:/Users/zen/Documents/-- four/s2/FYP I/XAl-on-healthcare-diagnostics/version_3.0/non-xai/decision_tree_model.pkl')

<font color="bluegrey" size=+1.0><b>Preprocess</b></font>

In [7]:
file_path = 'C:/Users/zen/Documents/-- four/s2/FYP I/XAl-on-healthcare-diagnostics/version_3.0/healthcare-dataset-stroke-data.csv'
data = pd.read_csv(file_path)

data['bmi'] = data['bmi'].fillna(data['bmi'].mean())

data = data.drop(columns=['id'])

target_column = 'stroke'

# Separate features and target
X = data.drop(columns=[target_column])
y = data[target_column]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

categorical_columns = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']
numerical_columns = ['age', 'avg_glucose_level', 'bmi']

X_train_processed, X_test_processed, pipeline = preprocess_data(X_train, X_test, categorical_columns, numerical_columns)

Processed Training Data Shape: (4088, 18)
Processed Testing Data Shape: (1022, 18)


<font color="bluegrey" size=+1.0><b>Logistic Regression</b></font>

In [ ]:
def predict_fn(X):
    return log_reg_model.predict_proba(X)[:, 1]

background_kmeans = shap.kmeans(X_train_processed, 50)  # Create 50 clusters
explainer_log = shap.KernelExplainer(predict_fn, background_kmeans)
shap_values_log = explainer_log.shap_values(X_test_processed)

shap.summary_plot(shap_values_log, X_test_processed, feature_names=X_test.columns)

c:\Users\zen\Documents\-- four\s2\FYP I\XAl-on-healthcare-diagnostics\xai\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
  0%|          | 0/1022 [00:00<?, ?it/s]c:\Users\zen\Documents\-- four\s2\FYP I\XAl-on-healthcare-diagnostics\xai\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\zen\Documents\-- four\s2\FYP I\XAl-on-healthcare-diagnostics\xai\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
  0%|          | 1/1022 [00:00<07:05,  2.40it/s]c:\Users\zen\Documents\-- four\s2\FYP I\XAl-on-healthcare-diagnostics\xai\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature nam

<font color="bluegrey" size=+1.0><b>Decision Tree</b></font>

In [ ]:
explainer_dt = shap.Explainer(dt_model, X_train_processed)

# Get SHAP values for the test set
shap_values_dt = explainer_dt(X_test_processed)

# Plot SHAP summary plot for Decision Tree
shap.summary_plot(shap_values_dt, X_test_processed, feature_names=X_test.columns)

# Visualize a single prediction's SHAP explanation for Decision Tree
shap.force_plot(shap_values_dt[0])  # Visualize the first prediction's explanation

In [ ]:
# Feature importance for Logistic Regression
shap.summary_plot(shap_values_log, X_test_processed, plot_type="bar")

# Feature importance for Decision Tree
shap.summary_plot(shap_values_dt, X_test_processed, plot_type="bar")